In [3]:
import csv

In [1]:
subjects = {
    1: ["male", 52],
    2: ["male", 40],
    3: ["female", 35],
    4: ["female", 44],
    5: ["male", 21],
    6: ["female", 21],
    7: ["male", 51],
    8: ["female", 51],
    9: ["male", 14],
    10: ["male", 23],
    11: ["female", 23],
    12: ["male", 52]
}

In [10]:
for i in range(1, 13):
    try:
        filename_drea = f"{str(i).zfill(2)}-drea.txt"
        filename_normal = f"{str(i).zfill(2)}-normal.txt"
        log_to_csv(filename_drea, filename_normal)
    except:
        print("Noch keine Daten")

Noch keine Daten


Fehler
Durchgang 02 im Modus drea 20.37 Minuten gedauert.
Durchgang 02 im Modus normal 16.13 Minuten gedauert.
Durchgang 03 im Modus drea 28.68 Minuten gedauert.
Durchgang 03 im Modus normal 21.89 Minuten gedauert.
Durchgang 04 im Modus drea 28.79 Minuten gedauert.
Durchgang 04 im Modus normal 15.43 Minuten gedauert.
Durchgang 05 im Modus drea 13.48 Minuten gedauert.
Durchgang 05 im Modus normal 12.59 Minuten gedauert.
Durchgang 06 im Modus drea 15.05 Minuten gedauert.
Durchgang 06 im Modus normal 13.62 Minuten gedauert.
Durchgang 07 im Modus drea 26.13 Minuten gedauert.
Durchgang 07 im Modus normal 13.65 Minuten gedauert.
Fehler
Fehler
Fehler
Fehler
Fehler


In [27]:
log_to_csv("03-drea.txt","06-normal.txt" )
#log_to_csv("06-normal.txt")

03-drea.txt
06-normal.txt


In [26]:
import pandas as pd

In [53]:
timedata = pd.read_csv("time.csv")
time_filter = timedata["time"] < 20
mode_filter = timedata["mode"] == "normal"

_filter = time_filter & mode_filter
timedata[_filter]

,id,mode,time,sex,age
1,2,normal,16.13,male,40
5,4,normal,15.43,female,44
7,5,normal,12.59,male,21
9,6,normal,13.62,female,21
11,7,normal,13.65,male,51
13,8,normal,18.59,female,51
15,9,normal,16.13,male,14
17,10,normal,12.13,male,23
19,11,normal,15.48,female,23
21,12,normal,18.21,male,52


In [ ]:
for i in range(2, 8):
    print("######################\n")
    drea = count_actions_for_word(f"0{i}-drea.csv")
    normal = count_actions_for_word(f"0{i}-normal.csv")
    drea_counter = 0
    normal_counter = 0
    for key, value in drea.items():
        drea_counter += drea[key]
        normal_counter += normal[key]
        #print(f"Wort: {key}; Drea Modus: {drea[key]}; Normaler Modus: {normal[key]}")
        
        print(f"Im DREA Modus waren es {normal[key] - drea[key]} weniger/mehr")

    print(f"\nInsgesamt waren es {drea_counter} Interaktionen im DREA Modus")
    print(f"Insgesamt waren es {normal_counter} Interaktionen im normal Modus\n")
    print("######################\n")

In [7]:
def calc_time(*filenames):
    start = 0
    end = 0
    for filename in filenames:
        with open(filename, "r") as file:
            for line in file:
                if "START" in line:
                    data = line.split(",")
                    start = int(data[1])
                if "END" in line: 
                    data = line.split(",")
                    end = int(data[1])
        time = (end - start) / 1000 / 60
        _filename = filename.split("-")
        _id = _filename[0]
        mode = _filename[1].split(".")[0]
        #print(f"Durchgang {person} im Modus {mode} {round(time, 2)} Minuten gedauert.")
        yield (_id, mode, round(time, 2))

In [25]:
with open("time.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["id", "mode", "time", "sex", "age"])
    for i in range(2, 13):
            try:
                filename_drea = f"{str(i).zfill(2)}-drea.csv"
                filename_normal = f"{str(i).zfill(2)}-normal.csv"
                for run in calc_time(filename_drea, filename_normal):
                    subject = list(run) + subjects[i]
                    writer.writerow(subject)

            except:
                print("Fehler")

In [8]:
# Verwandelt eine Log Datei in eine CSV Datei (vorher muss Fehler Log entfernt werden)
def log_to_csv(*filenames):  
    for filename in filenames:
        with open(filename, "r") as file:
            with open(filename.split(".txt")[0] + ".csv", "w", newline='') as csvfile:
                writer = csv.writer(csvfile, delimiter=",")
                writer.writerow(["event","timestamp","button","finger","word"])
                for line in file:
                    if "beginning" in line or "Proband" in line:
                        continue
                    data = line.split("{")[1].split("}")[0].split(",")
                    writer.writerow(data)

In [18]:
# Erzeugt ein Dict aus Worten und für das Wort notwendige Aktionen (kann verglichen werden)
def count_actions_for_word(filename):
    counter = 0
    old_word = None
    word_dict = dict()
    with open(filename, "r") as file:
        for line in file:
            # Wenn das nächste Wort angezeigt wird
            if "WORD_NEXT" in line:
                # Daten aufteilen
                data = line.split(",")
                # Test ob es das erste Wort ist
                if old_word == None:
                    old_word = data[4]
                    continue
                    
                old_word = data[4]
            if "WORD_DONE" in line:
                data = line.split(",")
                word_dict[old_word.strip()] = counter
                counter = 0
                    
            if "CHAR_SELECTED" in line or "CHAR_CLICKED" in line:
                counter = counter + 1

        return word_dict